In [ ]:
import numpy as np
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

In [ ]:
df_log = pd.read_csv("RMDemogFairnessData.csv")

In [ ]:
columns = ['num_of_items_purchased','num_of_problem_BC_filtered','num_of_hints']

In [ ]:
#RUN THIS - MATH SUCCESS
from scipy.stats import spearmanr
import math
import numpy as np
np.random.seed(0)
on = 'School_ID'#'District_ID'#'School_ID' 
vc = df_log[on].value_counts() 
school_or_dist = vc[vc >= 25].index
#list(df_log[on].value_counts().index)

cor_dict = {}
p_dict = {}
sample_size = {}
grade_list = {}
school_list = {}
student_list = []
for col in columns:
    st = 'problemA_accuracy'
    cor_dict[st+'_'+col] = []
    p_dict[st+'_'+col] = []
    sample_size[st+'_'+col] = []
    grade_list[st+'_'+col] = []
    school_list[st+'_'+col] = []

for val in school_or_dist:
    #filter on district/school
    df_hisp = df_log[df_log[on] == val]
    #print on, "= ", val 
    
    #grades = df_hisp['Grade'].unique() #school-grade level
    grades = [1] #school level
    for grade in grades:
        #filter on grade
        #df_hisp_grade = df_hisp[df_hisp['Grade'] == grade]#school-grade level
        df_hisp_grade = df_hisp #school level
    
        cor_list = []
        p_list = []
        #drop null entry students
        df_temp = df_hisp_grade[['problemA_accuracy', 'Student_ID'] + columns]
        df_temp.dropna(inplace=True)

        for col in columns:
            print col
            cor = spearmanr(df_temp[col], df_temp['problemA_accuracy'])
            print cor

            if math.isnan(cor[0]):
                #print cor
                continue
            else:
                student_list.append(list(df_temp['Student_ID']))
                cor_dict[st+'_'+col].append(cor[0])
                p_dict[st+'_'+col].append(cor[1])
                sample_size[st+'_'+col].append(len(df_temp))
                grade_list[st+'_'+col].append(grade)
                school_list[st+'_'+col].append(val)
                cor_list.append(cor[0])
                p_list.append(cor[1])

        #print st, np.mean(cor_list), np.mean(p_list)

In [ ]:
#RUN THIS - MATH IDENTITY
from scipy.stats import spearmanr
import math
import numpy as np
uniq_st = ['concept_pre','concept_post']

np.random.seed(0)
on = 'School_ID'#'District_ID'#'School_ID' 
vc = df_log[on].value_counts() 
school_or_dist = vc[vc >= 25].index
#list(df_log[on].value_counts().index)

cor_dict = {}
p_dict = {}
sample_size = {}
grade_list = {}
school_list = {}
student_list = {}
const_list = []
for st in uniq_st:
    for col in columns:
        const_list.append(st+'_'+col)
        cor_dict[st+'_'+col] = []
        p_dict[st+'_'+col] = []
        sample_size[st+'_'+col] = []
        grade_list[st+'_'+col] = []
        school_list[st+'_'+col] = []
        student_list[st+'_'+col] = []

for val in school_or_dist:
    #filter on district/school
    df_hisp = df_log[df_log[on] == val]
    #print on, "= ", val 
    
    #grades = df_hisp['Grade'].unique() #school-grade level
    grades = [1] #school level
    for grade in grades:
        #filter on grade
        #df_hisp_grade = df_hisp[df_hisp['Grade'] == grade]#school-grade level
        df_hisp_grade = df_hisp #school level
    
        #loop over construct
        for st in uniq_st:
            cor_list = []
            p_list = []
            #drop null entry students
            df_temp = df_hisp_grade[[st] + columns + ['Student_ID']]
            df_temp.dropna(inplace=True)
            if df_temp.shape[0] < 25: #number of students
                if st == uniq_st[0]:
                    print val
                continue
            #print "construct =", st
            #print len(df_temp)
            #print "\n"
            for col in columns:
                cor = spearmanr(df_temp[col], df_temp[st])
                if cor[1] < 0.05:
                    print col, cor
                if math.isnan(cor[0]):
                    #print cor
                    continue
                else:
                    student_list[st+'_'+col] = student_list[st+'_'+col] + list(df_temp['Student_ID'])
                    cor_dict[st+'_'+col].append(cor[0])
                    p_dict[st+'_'+col].append(cor[1])
                    sample_size[st+'_'+col].append(len(df_temp))
                    grade_list[st+'_'+col].append(grade)
                    school_list[st+'_'+col].append(val)
                    cor_list.append(cor[0])
                    p_list.append(cor[1])

            #print st, np.mean(cor_list), np.mean(p_list)

In [ ]:
outcome = 'problemA_accuracy'
#outcome = 'concept_pre'
#outcome = 'concept_post'

col = 'num_of_items_purchased'#'num_of_problem_BC_filtered'
const = outcome+'_'+col

print "number of students", sum(sample_size[const])

#FOR SUCCESS
df_studs_selected = df_log[df_log['School_ID'].isin(school_list[const])]

#FOR IDENTITY
#st_list = student_list[const]
#df_studs_selected = df_log[df_log['Student_ID'].isin(st_list)]   

print "number of students verify", len(df_studs_selected)

print "number of schools", len(school_list[const])


In [ ]:
#visualization
%matplotlib inline
#student-level
df_studs_selected[col].plot('box')
df_studs_selected[outcome].plot('box')
#school-level aggregates
df_studs_selected.groupby(['School_ID']).aggregate('mean')[col].plot('box')
df_studs_selected.groupby(['School_ID']).aggregate('mean')[outcome].plot('box')
#demogrpahics
#CPETECOP, CPETSPEP, CPETLEPP
df_studs_selected.groupby(['School_ID']).aggregate('mean')['CPETECOP'].plot('box')
#CPETHISP, CPETBLAP, CPETWHIP, CPETASIP
df_studs_selected.groupby(['School_ID']).aggregate('mean')['CPETWHIP'].plot('box')

In [ ]:
df_temp = df_studs_selected.groupby(['School_ID']).aggregate('mean')[['CPETHISP', 'CPETBLAP', 'CPETWHIP', 'CPETASIP']]
df_temp.columns = ['% H', '% AA', '% W', '% A']
df_temp.boxplot(fontsize='large')

In [ ]:
    df_schools = pd.DataFrame(school_list[construct])
    df_schools.columns = ['School_ID']
    df_schools['Grade'] = grade_list[construct]
    df_schools = df_schools.merge( df_log, left_on = "School_ID", right_on = "School_ID", how = "left")
    df_schools.drop(['School_ID'], axis=1, inplace=True)
 

In [ ]:
df_schools_full = df_log[['School_ID','CPETECOP','CPETLEPP','CPETSPEP', 'CPETHISP', 'CPETBLAP', 'CPETWHIP', 'CPETASIP']]
df_schools_full.drop_duplicates(inplace=True)
len(df_schools_full)

In [ ]:
import csv
row = []
row.append(['construct', 'demog', 'cor', 'p'])
demog_list = ['CPETECOP','CPETLEPP','CPETSPEP', 'CPETHISP', 'CPETBLAP', 'CPETWHIP', 'CPETASIP']
const_list = cor_dict.keys()
for construct in const_list:
    df_schools = df_schools_full[df_schools_full['School_ID'].isin(school_list[construct])]
    
    for demog in demog_list:
        if demog == 'CAMPUS' or demog == 'Unnamed: 0':
            continue
        cor = spearmanr(cor_dict[construct], df_schools[demog])
        #print construct, cor
        #if cor[1] <= 0.05:# and 'problemA' in construct:
        print construct, demog, cor[0], cor[1]
        row.append([construct, demog, cor[0], cor[1]])